In [1]:
# import required packages
import cv2
import argparse
import numpy as np
import matplotlib.pyplot as plt
import os
print(cv2. __version__)

4.5.5


In [2]:
# read yolo config 

# read class names from text file
classes = None
with open('...', 'r') as f: '''use class names here'''
    classes = [line.strip() for line in f.readlines()]

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))



# read pre-trained model and config file
net = cv2.dnn.readNet('...', 
                      '...')

In [3]:
# function to get the output layer names 
# in the architecture
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
#     output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

#### Video Capture

In [8]:
#Put Video here
cap = cv2.VideoCapture('...')

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:

        # Display the resulting frame
        cv2.imshow('Frame',frame)

        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

  # Break the loop
    else: 
        break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

#### Seatbelt Detection on Video

In [ ]:
## Put Video here
cap = cv2.VideoCapture('''...''')
# cap = cv2.VideoCapture('Camila Cabello Carpool Karaoke.mp4')


import tensorflow
model = tensorflow.keras.models.load_model('./....') ## Put custom classifier model weights here

video_size = (int(cap.get(3)),int(cap.get(4)))
video_fps = cap.get(cv2.CAP_PROP_FPS)
result = cv2.VideoWriter('inference.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         video_fps,video_size )
                    

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
    
        image = frame


        Width = image.shape[1]
        Height = image.shape[0]
        scale = 0.00392

        # create input blob 
        blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

        # set input blob for the network
        net.setInput(blob)

        outs = net.forward(get_output_layers(net))

        # initialization
        class_ids = []
        confidences = []
        boxes = []
        conf_threshold = 0.5
        nms_threshold = 0.4

        # for each detetion from each output layer 
        # get the confidence, class id, bounding box params
        # and ignore weak detections (confidence < 0.5)
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    w = int(detection[2] * Width)
                    h = int(detection[3] * Height)
                    x = center_x - w / 2
                    y = center_y - h / 2
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])

        # apply non-max suppression
        indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    #     print("boxes", boxes)
        print("indices", indices)



        # go through the detections remaining
        # after nms and draw bounding box
    #     for b in range(len(boxes)):
        for i in indices:
            idx = i[0]
            box = boxes[idx]
            x = abs(int(box[0]))
            y = abs(int(box[1]))
            w = abs(int(box[2]))
            h = abs(int(box[3]))

            if str(classes[class_ids[idx]]) == 'person' and h>80 and w>20:
                print(x,y,h,w)
                try:
                    temp_image = image[y:y+h, x:x+w]                    
                    temp_image = cv2.resize(temp_image, (416, 416))
                    temp_image = np.expand_dims(temp_image, axis = 0)/ 255.0
                    
                    
                    prediction = model.predict(temp_image)
                    label = 'person_seatbelt' if prediction[0][1] >= 0.5 else 'person_no_seatbelt'
                    cv2.rectangle(image,(x,y),(x+w,y+h),(255,255,255),1)
                    font = cv2.FONT_HERSHEY_PLAIN
                    cv2.putText(image,str(label),(x,y+30),font,1,(255,0,0),)
                    result.write(frame)
                except:
                    print("Skipped Frame")
                    pass
        cv2.imshow("object detection",image )
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    
    else:
        break
   # When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()